# LLaMA

In [ ]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!conda install -y pytorch::pytorch -c pytorch

/bin/bash: line 1: conda: command not found


In [ ]:
from google.colab import userdata
key = userdata.get('llama')

In [ ]:
from huggingface_hub import notebook_login
notebook_login(key)

In [ ]:
import transformers
import torch

# https://huggingface.co/meta-llama/Meta-Llama-3-8B
# 16GB이상의 VRAM 필요

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id= "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    # device_map="mps",
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


In [ ]:
pipeline.torch_dtype

torch.bfloat16

Llama3의 prompt format
- https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/

In [ ]:
messages = [
    {"role": "system", "content": "너는 도움이 되는 어시스턴트이다."},
    {"role": "user", "content": "잘가 영어로 뭐야?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
%%time
outputs = pipeline(
    messages,
    max_new_tokens=24,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


CPU times: user 2min 26s, sys: 3min 7s, total: 5min 33s
Wall time: 37min 22s


In [ ]:
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': 'You\'re asking what "잘가가" means in English! 😊\n\n"잘가가" is'}


In [ ]:
outputs

[{'generated_text': [{'role': 'system', 'content': '너는 도움이 되는 어시스턴트이다.'},
   {'role': 'user', 'content': '잘가가 영어로 뭐야?'},
   {'role': 'assistant',
    'content': 'You\'re asking what "잘가가" means in English! 😊\n\n"잘가가" is'}]}]

GPU를 장착한 Linux 머신이 있다면 더 빠른 추론을 위해 vllm LLM 서빙 엔진을 사용 할 수 있음
- vllm
  - https://github.com/vllm-project/vllm